In [4]:
import pandas as pd

data = pd.read_excel('RFM聚类分析【样本数据】.xlsx', index_col='用户编码')
data.head()

,首次投资时间,最近一次投资时间,总计投资总金额,总计投标总次数
用户编码,,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,2016-06-25,2016-06-26,20000.0,3
9921C535-75FC-4695-8551-32B8D717B4A5,2016-06-26,2016-06-26,50000.0,3
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,2016-06-13,2016-06-19,108000.0,4
150218D8-BF32-406D-A3A8-8A19CB1E85AA,2016-06-16,2016-06-16,30000.0,3
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,2016-06-27,2016-06-27,100000.0,3


In [5]:
len(data)

200

## 数据探索及预处理

In [10]:
data.describe(include='all')

,首次投资时间,最近一次投资时间,总计投资总金额,总计投标总次数
count,200,200,200.000000,200.00000
unique,24,38,NaN,NaN
top,2016-06-22 00:00:00,2016-06-25 00:00:00,NaN,NaN
freq,20,25,NaN,NaN
first,2016-06-06 00:00:00,2016-06-06 00:00:00,NaN,NaN
last,2016-06-29 00:00:00,2016-07-18 00:00:00,NaN,NaN
mean,NaN,NaN,47223.409500,5.92000
std,NaN,NaN,43381.496372,4.84883
min,NaN,NaN,10100.000000,2.00000
25%,NaN,NaN,20000.000000,3.00000


In [13]:
from datetime import datetime

exdata_date = datetime(2016,7,20)
print(exdata_date)

2016-07-20 00:00:00


In [18]:
# R最近一次投资时间据提数日天数
diff_R = exdata_date-data['最近一次投资时间']
diff_R.head()

用户编码
45B3CCE7-957B-4D54-9626-6D62731D119B   24 days
9921C535-75FC-4695-8551-32B8D717B4A5   24 days
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8   31 days
150218D8-BF32-406D-A3A8-8A19CB1E85AA   34 days
98C26D4A-708F-4334-B3D7-EDAE9FD240C8   23 days
Name: 最近一次投资时间, dtype: timedelta64[ns]

In [20]:
diff_R[0].days

24

In [21]:
type(diff_R[0].days)

int

In [22]:
R = []
for i in diff_R:
    R.append(i.days)

R[:10]

[24, 24, 31, 34, 23, 28, 39, 22, 38, 25]

In [23]:
'''
用户在投时长（月）
1、用户在投时长（天）
2、月=天/30,向上取整
'''
diff = exdata_date-data['首次投资时间']
diff.head()

用户编码
45B3CCE7-957B-4D54-9626-6D62731D119B   25 days
9921C535-75FC-4695-8551-32B8D717B4A5   24 days
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8   37 days
150218D8-BF32-406D-A3A8-8A19CB1E85AA   34 days
98C26D4A-708F-4334-B3D7-EDAE9FD240C8   23 days
Name: 首次投资时间, dtype: timedelta64[ns]

In [25]:
diff[0].days/30

0.8333333333333334

In [26]:
# ceil ,向上取整
from math import ceil

ceil(diff[0].days/30)

1

In [27]:
# 获取相差天数
diff_days = []
for i in diff:
    diff_days.append(i.days)

diff_days[:10]


[25, 24, 37, 34, 23, 28, 41, 22, 38, 25]

In [28]:
# 通过循环向上取整，得到用户在投时长（月）
diff_months = []

for i in diff_days:
    diff_months.append(ceil(i/30))

diff_months[:10]

[1, 1, 2, 2, 1, 1, 2, 1, 2, 1]

In [30]:
#F, 月均投资次数
F=data['总计投标总次数']/diff_months
F.head()

用户编码
45B3CCE7-957B-4D54-9626-6D62731D119B    3.0
9921C535-75FC-4695-8551-32B8D717B4A5    3.0
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8    2.0
150218D8-BF32-406D-A3A8-8A19CB1E85AA    1.5
98C26D4A-708F-4334-B3D7-EDAE9FD240C8    3.0
Name: 总计投标总次数, dtype: float64

In [32]:
F.values[:10]

array([3. , 3. , 2. , 1.5, 3. , 3. , 1.5, 3. , 1. , 3. ])

In [36]:
# M, 月均投资金额
M = (data['总计投资总金额']/diff_months).values
M[:10]

array([ 20000.,  50000.,  54000.,  15000., 100000.,  15000.,  10000.,
        16000.,   7500.,  20000.])

In [42]:
# 选取 R、F、M 三个指标作为聚类分析指标
from pandas import DataFrame

cdata=DataFrame([R,list(F),list(M)]).T
cdata.head()

,0,1,2
0,24.0,3.0,20000.0
1,24.0,3.0,50000.0
2,31.0,2.0,54000.0
3,34.0,1.5,15000.0
4,23.0,3.0,100000.0


In [44]:
#指定cdata的index,columns
cdata.index = data.index
cdata.columns = ['R-最近一次投资时间据提数日的天数', 'F-月均投资次数', 'M-月均投资金额']
cdata.head()

,R-最近一次投资时间据提数日的天数,F-月均投资次数,M-月均投资金额
用户编码,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,24.0,3.0,20000.0
9921C535-75FC-4695-8551-32B8D717B4A5,24.0,3.0,50000.0
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,31.0,2.0,54000.0
150218D8-BF32-406D-A3A8-8A19CB1E85AA,34.0,1.5,15000.0
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,23.0,3.0,100000.0


In [45]:
cdata.mean()

R-最近一次投资时间据提数日的天数       22.66500
F-月均投资次数                 4.24250
M-月均投资金额             35821.24175
dtype: float64

In [46]:
zcdata=(cdata-cdata.mean())/cdata.std()
zcdata.head()

,R-最近一次投资时间据提数日的天数,F-月均投资次数,M-月均投资金额
用户编码,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,0.140422,-0.353751,-0.392133
9921C535-75FC-4695-8551-32B8D717B4A5,0.140422,-0.353751,0.351423
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,0.876714,-0.638461,0.450564
150218D8-BF32-406D-A3A8-8A19CB1E85AA,1.192268,-0.780815,-0.516059
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,0.035237,-0.353751,1.590683


In [49]:
#sklearn
from sklearn.cluster import KMeans

kmodel=KMeans(n_clusters=4,n_jobs=4,max_iter=100,random_state=0)
kmodel.fit(zcdata)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=4, n_init=10, n_jobs=4, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [50]:
kmodel.labels_

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0,
       0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 2, 0, 1, 0, 2, 0, 0, 2, 3, 2,
       2, 2, 2, 0, 1, 0, 2, 0, 2, 2, 2, 0, 1, 0, 3, 1, 0, 2, 2, 0, 0, 0,
       2, 0, 0, 0, 2, 2, 3, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       3, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 1, 0, 2, 0,
       3, 2, 2, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 1, 2, 1,
       2, 0, 2, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2,
       0, 0], dtype=int32)

In [51]:
#统计每个类别的频率

from pandas import Series

Series(kmodel.labels_).value_counts()

0    128
2     44
1     19
3      9
dtype: int64

In [55]:
# 将类别标签赋回原数据
cdata_rst = pd.concat([cdata, Series(kmodel.labels_,index=cdata.index)],axis=1)
cdata_rst.head()

,R-最近一次投资时间据提数日的天数,F-月均投资次数,M-月均投资金额,0
用户编码,,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,24.0,3.0,20000.0,0
9921C535-75FC-4695-8551-32B8D717B4A5,24.0,3.0,50000.0,0
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,31.0,2.0,54000.0,0
150218D8-BF32-406D-A3A8-8A19CB1E85AA,34.0,1.5,15000.0,0
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,23.0,3.0,100000.0,1


In [56]:
#命名最后一列为类别
cdata_rst.columns=list(cdata.columns)+['类别']
cdata_rst.head()

,R-最近一次投资时间据提数日的天数,F-月均投资次数,M-月均投资金额,类别
用户编码,,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,24.0,3.0,20000.0,0
9921C535-75FC-4695-8551-32B8D717B4A5,24.0,3.0,50000.0,0
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,31.0,2.0,54000.0,0
150218D8-BF32-406D-A3A8-8A19CB1E85AA,34.0,1.5,15000.0,0
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,23.0,3.0,100000.0,1


In [57]:
#按照类别分组统计R、F、M指标的均值
#groupby
cdata_rst.groupby(cdata_rst['类别']).mean()

,R-最近一次投资时间据提数日的天数,F-月均投资次数,M-月均投资金额
类别,,,
0,27.859375,2.820312,21906.754297
1,20.684211,4.552632,115842.105263
2,10.568182,5.579545,26984.313636
3,12.111111,17.277778,107986.000000


**结论：**

- 类别3： R、F、M都比较高，数据重要价值客户，或者超级用户
- 类别0： R、F、M都比较低，属于低价值客户
- 类别1： M很高，F还行, R不高，保持客户